In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build the model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# Predict on test data
predictions = model.predict(x_test)

# Plot the first test image and its prediction
plt.imshow(x_test[0], cmap=plt.cm.binary)
plt.title(f"Predicted: {tf.argmax(predictions[0]).numpy()}, Actual: {y_test[0]}")
plt.show()

2025-03-29 17:54:59.292124: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 17:54:59.898100: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 17:55:00.253861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743270900.736874   29255 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743270900.867428   29255 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743270901.587034   29255 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

: 

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample data for chatbot (input-output pairs)
input_texts = ["hello", "how are you", "what is your name", "bye"]
target_texts = ["hi", "I am fine, thank you", "I am a chatbot", "goodbye"]

# Tokenize the data
input_characters = sorted(set("".join(input_texts)))
target_characters = sorted(set("".join(target_texts)))

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, len(input_characters)), dtype="float32")
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, len(target_characters)), dtype="float32")
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, len(target_characters)), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# Build the model
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None, len(input_characters)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, len(target_characters)))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(target_characters), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train the model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100)

# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Decode function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(target_characters)))
    target_seq[0, 0, target_token_index[" "]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = list(target_token_index.keys())[list(target_token_index.values()).index(sampled_token_index)]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, len(target_characters)))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence

# Test the chatbot
test_input = "hello"
test_input_seq = np.zeros((1, max_encoder_seq_length, len(input_characters)), dtype="float32")
for t, char in enumerate(test_input):
    test_input_seq[0, t, input_token_index[char]] = 1.0

decoded_sentence = decode_sequence(test_input_seq)
print(f"Chatbot response: {decoded_sentence}")

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

import torch.nn as nn
import torch.optim as optim

# Define the dataset
class ChatDataset(Dataset):
    def __init__(self, input_texts, target_texts, input_token_index, target_token_index, max_input_len, max_target_len):
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.input_token_index = input_token_index
        self.target_token_index = target_token_index
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_seq = torch.zeros(self.max_input_len, len(self.input_token_index))
        target_seq = torch.zeros(self.max_target_len, len(self.target_token_index))

        for t, char in enumerate(self.input_texts[idx]):
            input_seq[t, self.input_token_index[char]] = 1.0
        for t, char in enumerate(self.target_texts[idx]):
            target_seq[t, self.target_token_index[char]] = 1.0

        return input_seq, target_seq

# Define the encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, x):
        _, (hidden, cell) = self.lstm(x)
        return hidden, cell

# Define the decoder
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(output_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        output = self.fc(output)
        return output, hidden, cell

# Define the Seq2Seq model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, target_seq, teacher_forcing_ratio=0.5):
        batch_size, target_len, _ = target_seq.size()
        outputs = torch.zeros_like(target_seq)

        hidden, cell = self.encoder(input_seq)
        decoder_input = target_seq[:, 0, :].unsqueeze(1)

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[:, t, :] = output.squeeze(1)
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            decoder_input = target_seq[:, t, :].unsqueeze(1) if teacher_force else output

        return outputs

# Hyperparameters
input_size = len(input_token_index)
output_size = len(target_token_index)
hidden_size = 256
batch_size = 16
epochs = 100

# Prepare the dataset and dataloader
dataset = ChatDataset(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
encoder = Encoder(input_size, hidden_size)
decoder = Decoder(output_size, hidden_size)
model = Seq2Seq(encoder, decoder)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(epochs):
    for input_seq, target_seq in dataloader:
        optimizer.zero_grad()
        output = model(input_seq, target_seq)
        loss = criterion(output.view(-1, output_size), target_seq.view(-1, output_size))
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Test the chatbot
def predict(input_text):
    input_seq = torch.zeros(1, max_encoder_seq_length, len(input_token_index))
    for t, char in enumerate(input_text):
        input_seq[0, t, input_token_index[char]] = 1.0

    hidden, cell = encoder(input_seq)
    decoder_input = torch.zeros(1, 1, len(target_token_index))
    decoder_input[0, 0, target_token_index[" "]] = 1.0

    decoded_sentence = ""
    for _ in range(max_decoder_seq_length):
        output, hidden, cell = decoder(decoder_input, hidden, cell)
        sampled_token_index = output.argmax(2).item()
        sampled_char = list(target_token_index.keys())[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n":
            break
        decoder_input = torch.zeros(1, 1, len(target_token_index))
        decoder_input[0, 0, sampled_token_index] = 1.0

    return decoded_sentence

test_input = "hello"
response = predict(test_input)
print(f"Chatbot response: {response}")

In [ ]:
from flask import Flask, render_template, Response, request, send_file
import os

app = Flask(__name__)

# Path to the directory containing video files
VIDEO_DIR = "videos"

@app.route('/')
def index():
    # List all video files in the directory
    videos = os.listdir(VIDEO_DIR)
    return render_template('index.html', videos=videos)

@app.route('/video/<filename>')
def video(filename):
    # Serve the video file
    return send_file(os.path.join(VIDEO_DIR, filename))

if __name__ == '__main__':
    # Ensure the video directory exists
    if not os.path.exists(VIDEO_DIR):
        os.makedirs(VIDEO_DIR)
    app.run(debug=True)

In [ ]:
from flask import Flask, request, render_template

app = Flask(__name__)

# Sample dataset
data = {
    "Google": "Google is a search engine.",
    "Python": "Python is a programming language.",
    "Flask": "Flask is a web framework for Python.",
    "AI": "AI stands for Artificial Intelligence."
}

@app.route('/')
def index():
    return render_template('search.html')

@app.route('/search', methods=['GET'])
def search():
    query = request.args.get('q', '').lower()
    results = {key: value for key, value in data.items() if query in key.lower() or query in value.lower()}
    return render_template('results.html', query=query, results=results)

if __name__ == '__main__':
    app.run(debug=True)